# Fragment ion intensities Prediction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This is an extension of the original walkthrough example available [here](https://github.com/wilhelm-lab/dlomix-resources/tree/tasks/intensity/notebooks/Intensity/Example_IntensityModel_Walkthrough_colab.ipynb).

### Task 3: Calibration
Similar to the initial notebook, we will initialize our model and train it. The target here is to experiment with different feature calibrations and observe the impact on the performance and whether it reflects a realistic evaluation. 

In [3]:
import numpy
import functools
import dlomix.losses as losses
import tensorflow as tf


def reshape_dims(array):
    n, dims = array.shape
    assert dims == 174
    nlosses = 1
    return array.reshape(
        [array.shape[0], 30 - 1, 2, nlosses, 3]
    )


def reshape_flat(array):
    s = array.shape
    flat_dim = [s[0], functools.reduce(lambda x, y: x * y, s[1:], 1)]
    return array.reshape(flat_dim)


def normalize_base_peak(array):
    # flat
    maxima = array.max(axis=1)
    array = array / maxima[:, numpy.newaxis]
    return array


def mask_outofrange(array, lengths, mask=-1.):
    # dim
    for i in range(array.shape[0]):
        array[i, lengths[i] - 1 :, :, :, :] = mask
    return array


def mask_outofcharge(array, charges, mask=-1.):
    # dim
    for i in range(array.shape[0]):
        if charges[i] < 3:
            array[i, :, :, :, charges[i] :] = mask
    return array


def get_spectral_angle(true, pred, batch_size=600):

    n = true.shape[0]
    sa = numpy.zeros([n])

    def iterate():
        if n > batch_size:
            for i in range(n // batch_size):
                true_sample = true[i * batch_size : (i + 1) * batch_size]
                pred_sample = pred[i * batch_size : (i + 1) * batch_size]
                yield i, true_sample, pred_sample
            i = n // batch_size
            yield i, true[(i) * batch_size :], pred[(i) * batch_size :]
        else:
            yield 0, true, pred

    for i, t_b, p_b in iterate():
        tf.compat.v1.reset_default_graph()
        with tf.compat.v1.Session() as s:
            sa_graph = losses.masked_spectral_distance(t_b, p_b)
            sa_b = 1 - s.run(sa_graph)
            sa[i * batch_size : i * batch_size + sa_b.shape[0]] = sa_b
    sa = numpy.nan_to_num(sa)
    return sa


def normalize_predictions(data, batch_size=600):
    assert "sequences" in data
    assert "intensities_pred" in data
    assert "precursor_charge_onehot" in data

    sequence_lengths = data["sequences"].apply(lambda x: len(x))
    intensities =  np.stack(data["intensities_pred"].to_numpy()).astype(np.float32)
    precursor_charge_onehot = np.stack(data["precursor_charge_onehot"].to_numpy())
    charges = list(precursor_charge_onehot.argmax(axis=1) + 1)

    intensities[intensities < 0] = 0
    intensities = reshape_dims(intensities)
    intensities = mask_outofrange(intensities, sequence_lengths)
    intensities = mask_outofcharge(intensities, charges)
    intensities = reshape_flat(intensities)
    m_idx = intensities == -1
    intensities = normalize_base_peak(intensities)
    intensities[m_idx] = -1
    data["intensities_pred"] = intensities

    if "intensities_raw" in data:
        data["spectral_angle"] = get_spectral_angle(
            np.stack(data["intensities_raw"].to_numpy()).astype(np.float32), intensities, batch_size=batch_size
        )
    return data


In [1]:
# install the mlomix package in the current environment using pip

!python -m pip install -q git+https://github.com/wilhelm-lab/dlomix.git@feature/intensity_tutorial

In [ ]:
!python -m pip install wandb

In [16]:
import numpy as np
import pandas as pd
import dlomix
from dlomix.models import PrositIntensityPredictor
import tensorflow as tf
from dlomix.losses import masked_spectral_distance, pearson_correlation
tf.get_logger().setLevel('ERROR')

import wandb
from wandb.keras import WandbCallback

In [2]:
# enter project name for weights and biases
project_name = 'dlomix_intensity'

The code below creates a dataset, creates the model, and trains it. You should try with the two available features (`feature_a` and `feature_b`). Please Refer to the initial notebook to analyze the results.

Hint: Use the paths available below. Description for splits is as follows:
- suffix `_A`: feature A
- suffix `_B`: feature B

In [8]:
TRAIN_DATAPATH_A = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/feature_a/proteomeTools_train_val_a.csv'
TRAIN_DATAPATH_B = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/feature_b/proteomeTools_train_val_b.csv'

TEST_DATAPATH_A = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/feature_a/proteomeTools_test_a.csv'
TEST_DATAPATH_B = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/tasks/intensity/example_datasets/Intensity/feature_b/proteomeTools_test_b.csv'

In [11]:
from dlomix.data import IntensityDataset

BATCH_SIZE = 128

int_data = IntensityDataset(data_source=TRAIN_DATAPATH_A,
                              seq_length=30, collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2)


# this is the test dataset object, do not forget to change it to the respective suffix (A or B)
# when you change the training dataset

test_int_data = IntensityDataset(data_source=TEST_DATAPATH_A,
                              seq_length=30, collision_energy_col='collision_energy', batch_size=BATCH_SIZE, test=True)

<class 'pandas.core.series.Series'>
0        [0, 1, 0, 0, 0, 0]
1        [0, 0, 1, 0, 0, 0]
2        [0, 1, 0, 0, 0, 0]
3        [1, 0, 0, 0, 0, 0]
4        [0, 1, 0, 0, 0, 0]
                ...        
66249    [0, 1, 0, 0, 0, 0]
66250    [0, 1, 0, 0, 0, 0]
66251    [0, 1, 0, 0, 0, 0]
66252    [0, 1, 0, 0, 0, 0]
66253    [0, 1, 0, 0, 0, 0]
Name: precursor_charge_onehot, Length: 66254, dtype: object
<class 'pandas.core.series.Series'>
0        [0, 1, 0, 0, 0, 0]
1        [0, 1, 0, 0, 0, 0]
2        [0, 1, 0, 0, 0, 0]
3        [0, 1, 0, 0, 0, 0]
4        [0, 0, 0, 0, 1, 0]
                ...        
16559    [0, 0, 0, 1, 0, 0]
16560    [1, 0, 0, 0, 0, 0]
16561    [0, 1, 0, 0, 0, 0]
16562    [0, 1, 0, 0, 0, 0]
16563    [0, 1, 0, 0, 0, 0]
Name: precursor_charge_onehot, Length: 16564, dtype: object


In [17]:
# Enter weights and biases run name. Make sure that different features datasets have different names.
wandb.init(project=project_name, name='features_a')

# create model
model = PrositIntensityPredictor(seq_length=30)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom time-delta metric

model.compile(optimizer=optimizer, 
            loss=masked_spectral_distance, metrics=[pearson_correlation,'mean_absolute_error', 'mse'])

history = model.fit(int_data.train_data, validation_data=int_data.val_data, epochs=15,
                    callbacks=[WandbCallback(save_model=False)])


# Mark the run as finished
wandb.finish()

Consider rewriting this model with the Functional API.
encoded sequence:  Tensor("prosit_intensity_predictor_2/string_lookup_2/Identity:0", shape=(None, 30), dtype=int64)
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
encoded sequence:  Tensor("prosit_intensity_predictor_2/string_lookup_2/Identity:0", shape=(None, 30), dtype=int64)
Consider rewriting this model with the Functional API.
104/415 [======>.......................] - ETA: 4:25 - loss: 0.6946 - mean_absolute_error: 31.1215

KeyboardInterrupt: 

### Bonus:
After analyzing the results, different are these features from each other?